In [1]:
import nltk
import numpy as np
import re
from nltk.corpus import cmudict
#http://www.nltk.org/api/nltk.corpus.reader.html#module-nltk.corpus.reader.cmudict

In [2]:
ARPAbet = ['AA', 'AE', 'AH', 'AO', 'AW', 'AY', 'B', 'CH', 'D', 'DH', 'EH', 'ER', 'EY', 'F', 'G', 'HH', 'IH', 'IY', 'JH', 'K', 'L', 'M', 'N', 'NG', 'OW', 'OY', 'P', 'R', 'S', 'SH', 'T', 'TH', 'UH', 'UW', 'V', 'W', 'Y', 'Z', 'ZH']
arpa = {}
for i in range(len(ARPAbet)):
    arpa[ARPAbet[i]] = i
print len(ARPAbet)

39


In [3]:
d = cmudict.dict()
e = cmudict.entries()
x_train = []
y_train = []
for x in e:
    #print x
    w = x[0]
    skip = False
    for c in w:
        if ord(c) < ord('a') or ord(c) > ord('z'):
            skip = True
    if skip:
        continue
    w.encode('ascii','ignore')
    w = list(w)
    x_train.append(w)
    stress = -1
    for y in x[1]:
        y.encode('ascii','ignore')
        if(y[len(y) - 1] == '1'):
            stress = arpa[y[0:-1]]
    y_train.append(stress)
maxlen = 35
X = np.zeros((len(x_train), maxlen, 26), dtype=np.bool)
y = np.zeros((len(y_train), len(ARPAbet)), dtype=np.bool)
for i, word in enumerate(x_train):
    for t, char in enumerate(word):
        #print word
        X[i, t, ord(char) - ord('a')] = 1
    y[i, y_train[i]] = 1


In [4]:
print d["goodbye"]
print e[110]
print x_train[11101]
c = 0
for x in e:
    c = max(c, len(x[0]))
print c
#print X[1000]

[[u'G', u'UH2', u'D', u'B', u'AY1']]
(u'abducted', [u'AH0', u'B', u'D', u'AH1', u'K', u'T', u'IH0', u'D'])
[u'b', u'l', u'a', u'c', u'k', u'o', u'u', u't', u's']
33


In [17]:
# Build the model.
from keras.models import Sequential
from keras.layers.core import TimeDistributedDense, Dropout, Activation, Dense
from keras.layers.recurrent import LSTM
from keras.optimizers import RMSprop, adam
model = Sequential()
model.add(LSTM(100, input_shape=(maxlen, 26), return_sequences=True))
model.add(LSTM(100))
model.add(Dense(len(ARPAbet), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=adam(), metrics=['accuracy'])
print(model.summary())
#model.load_weights('model.h5')

____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
lstm_8 (LSTM)                      (None, 35, 100)     50800       lstm_input_6[0][0]               
____________________________________________________________________________________________________
lstm_9 (LSTM)                      (None, 100)         80400       lstm_8[0][0]                     
____________________________________________________________________________________________________
dense_6 (Dense)                    (None, 39)          3939        lstm_9[0][0]                     
Total params: 135139
____________________________________________________________________________________________________
None


In [ ]:
model.load_weights('model.h5')
model.fit(X, y, batch_size=100, nb_epoch=15)
#model.save_weights('model.h5', overwrite=True)

In [11]:
#model.load_weights('finalweights.h5')
def stress(w):
    w.encode('ascii','ignore')
    w = list(w)
    X = np.zeros((1, maxlen, 26), dtype=np.bool)
    for t, char in enumerate(w):
        X[0, t, ord(char) - ord('a')] = 1
    m = model.predict(X)
    #print X
    a = np.argmax(m)
    print a
    return(ARPAbet[a])
print stress('stress')
print 

10
EH



In [16]:
model.save_weights('model.h5', overwrite=True)